In [165]:
from pandas import DataFrame, read_csv

In [166]:
columns = {
    'HORA': 'checktime',  # date and time
    'FECHA': 'date',  # date
    'TARJETA': 'dni',  # identification number
    'Nombre': 'name',  # name
}

In [167]:
# use encoding for spanish characters: 
# only read the columns we need
# and format the date and time columns as dates
df = read_csv('ATTLOG.csv', dtype=str, encoding='latin-1', usecols=columns.keys(), parse_dates=['HORA', 'FECHA'],
              dayfirst=True)

In [168]:
# rename the columns
df.rename(columns=columns, inplace=True)

In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16756 entries, 0 to 16755
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   checktime  16756 non-null  object        
 1   date       16756 non-null  datetime64[ns]
 2   name       16756 non-null  object        
 3   dni        16756 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 523.8+ KB


In [170]:
df.head()

,checktime,date,name,dni
0,28/10/2022 12:32 p.m.,2022-10-28,48284572,48284572
1,09/11/2022 12:35 p.m.,2022-11-09,41303173,41303173
2,09/11/2022 12:41 p.m.,2022-11-09,"Espejo Paz, Paul Jhonnat",42371480
3,09/11/2022 01:04 p.m.,2022-11-09,41303173,41303173
4,11/11/2022 10:47 a.m.,2022-11-11,48284572,48284572


In [171]:
# remove the rows with missing values
df.dropna(inplace=True)

In [172]:
# fix 'dni' column
# must be 8 characters long
# if has less than 8 characters, add zeros at the beginning
df['dni'] = df['dni'].apply(lambda x: x.zfill(8))

In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16756 entries, 0 to 16755
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   checktime  16756 non-null  object        
 1   date       16756 non-null  datetime64[ns]
 2   name       16756 non-null  object        
 3   dni        16756 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 523.8+ KB


In [174]:
df.head()

,checktime,date,name,dni
0,28/10/2022 12:32 p.m.,2022-10-28,48284572,48284572
1,09/11/2022 12:35 p.m.,2022-11-09,41303173,41303173
2,09/11/2022 12:41 p.m.,2022-11-09,"Espejo Paz, Paul Jhonnat",42371480
3,09/11/2022 01:04 p.m.,2022-11-09,41303173,41303173
4,11/11/2022 10:47 a.m.,2022-11-11,48284572,48284572


In [175]:
from datetime import datetime

# define date range: from last december to this month
start_date = datetime(2023, 12, 1)
end_date = datetime.now()

In [176]:
# filter the dataframe by date range
df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1573 entries, 15183 to 16755
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   checktime  1573 non-null   object        
 1   date       1573 non-null   datetime64[ns]
 2   name       1573 non-null   object        
 3   dni        1573 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 61.4+ KB


In [178]:
df.head()

,checktime,date,name,dni
15183,01/12/2023 06:12 a.m.,2023-12-01,"Cachique Saldaña, Trinid",00080403
15184,01/12/2023 08:20 a.m.,2023-12-01,"Valdivia Novoa, Segundo",26646074
15185,01/12/2023 09:03 a.m.,2023-12-01,"Poma Vilca, Jimmy Christ",41011338
15186,01/12/2023 09:06 a.m.,2023-12-01,"Leon Gaspar, Evelyn",76753084
15187,01/12/2023 09:20 a.m.,2023-12-01,43084834,43084834


In [179]:
# get only employees in dni list    
dni_list = ['06927369', '40664656']

In [180]:
df = df[df['dni'].isin(dni_list)]

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71 entries, 15241 to 16736
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   checktime  71 non-null     object        
 1   date       71 non-null     datetime64[ns]
 2   name       71 non-null     object        
 3   dni        71 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 2.8+ KB


In [182]:
df.head()

,checktime,date,name,dni
15241,04/12/2023 03:20 p.m.,2023-12-04,"Romero Leon, Maria Anton",06927369
15273,04/12/2023 11:04 p.m.,2023-12-04,"Romero Leon, Maria Anton",06927369
15315,05/12/2023 03:19 p.m.,2023-12-05,"Romero Leon, Maria Anton",06927369
15346,05/12/2023 10:42 p.m.,2023-12-05,"Romero Leon, Maria Anton",06927369
15411,06/12/2023 10:56 p.m.,2023-12-06,"Romero Leon, Maria Anton",06927369


In [183]:
# sort by dni and date ASC
df.sort_values(['dni', 'date'], inplace=True)

In [184]:
# create two new columns: checkin and checkout
# donde checkin is the first checktime of the day
# and checkout is the last checktime of the day
# pero a veces el empleado no marca la salida y queda solo el checkin 
# por lo que el checkout seria null
df['checkin'] = df.groupby(['dni', 'date'])['checktime'].transform('min')
df['checkout'] = df.groupby(['dni', 'date'])['checktime'].transform('max')

# si el checkout es igual al checkin, entonces el empleado no marco la salida
# y el checkout es null
df.loc[df['checkin'] == df['checkout'], 'checkout'] = None

In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71 entries, 15241 to 16726
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   checktime  71 non-null     object        
 1   date       71 non-null     datetime64[ns]
 2   name       71 non-null     object        
 3   dni        71 non-null     object        
 4   checkin    71 non-null     object        
 5   checkout   66 non-null     object        
dtypes: datetime64[ns](1), object(5)
memory usage: 3.9+ KB


In [186]:
df.head()

,checktime,date,name,dni,checkin,checkout
15241,04/12/2023 03:20 p.m.,2023-12-04,"Romero Leon, Maria Anton",06927369,04/12/2023 03:20 p.m.,04/12/2023 11:04 p.m.
15273,04/12/2023 11:04 p.m.,2023-12-04,"Romero Leon, Maria Anton",06927369,04/12/2023 03:20 p.m.,04/12/2023 11:04 p.m.
15315,05/12/2023 03:19 p.m.,2023-12-05,"Romero Leon, Maria Anton",06927369,05/12/2023 03:19 p.m.,05/12/2023 10:42 p.m.
15346,05/12/2023 10:42 p.m.,2023-12-05,"Romero Leon, Maria Anton",06927369,05/12/2023 03:19 p.m.,05/12/2023 10:42 p.m.
15411,06/12/2023 10:56 p.m.,2023-12-06,"Romero Leon, Maria Anton",06927369,06/12/2023 10:56 p.m.,None


In [187]:
# como ya tenemos el checkin y checkout, podemos eliminar los duplicados basados en dni y date
# ya que un empleado no puede marcar mas de una vez en el mismo dia
df.drop_duplicates(['dni', 'date'], inplace=True)

In [188]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 15241 to 16726
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   checktime  38 non-null     object        
 1   date       38 non-null     datetime64[ns]
 2   name       38 non-null     object        
 3   dni        38 non-null     object        
 4   checkin    38 non-null     object        
 5   checkout   33 non-null     object        
dtypes: datetime64[ns](1), object(5)
memory usage: 2.1+ KB


In [189]:
df.head()

,checktime,date,name,dni,checkin,checkout
15241,04/12/2023 03:20 p.m.,2023-12-04,"Romero Leon, Maria Anton",06927369,04/12/2023 03:20 p.m.,04/12/2023 11:04 p.m.
15315,05/12/2023 03:19 p.m.,2023-12-05,"Romero Leon, Maria Anton",06927369,05/12/2023 03:19 p.m.,05/12/2023 10:42 p.m.
15411,06/12/2023 10:56 p.m.,2023-12-06,"Romero Leon, Maria Anton",06927369,06/12/2023 10:56 p.m.,None
15483,11/12/2023 03:22 p.m.,2023-12-11,"Romero Leon, Maria Anton",06927369,11/12/2023 03:22 p.m.,11/12/2023 11:51 p.m.
15556,12/12/2023 03:14 p.m.,2023-12-12,"Romero Leon, Maria Anton",06927369,12/12/2023 03:14 p.m.,12/12/2023 11:42 p.m.


In [190]:
# ahora podemos eliminar la columna checktime
df.drop('checktime', axis=1, inplace=True)

In [191]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 15241 to 16726
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      38 non-null     datetime64[ns]
 1   name      38 non-null     object        
 2   dni       38 non-null     object        
 3   checkin   38 non-null     object        
 4   checkout  33 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 1.8+ KB


In [192]:
df.head()

,date,name,dni,checkin,checkout
15241,2023-12-04,"Romero Leon, Maria Anton",06927369,04/12/2023 03:20 p.m.,04/12/2023 11:04 p.m.
15315,2023-12-05,"Romero Leon, Maria Anton",06927369,05/12/2023 03:19 p.m.,05/12/2023 10:42 p.m.
15411,2023-12-06,"Romero Leon, Maria Anton",06927369,06/12/2023 10:56 p.m.,None
15483,2023-12-11,"Romero Leon, Maria Anton",06927369,11/12/2023 03:22 p.m.,11/12/2023 11:51 p.m.
15556,2023-12-12,"Romero Leon, Maria Anton",06927369,12/12/2023 03:14 p.m.,12/12/2023 11:42 p.m.


In [193]:
from pandas import NaT

In [194]:
# check if 'checkin' and 'checkout' are datetime
print(df['checkin'].dtype)
# fill the missing values with NaT
df['checkout'].fillna(NaT, inplace=True)

object


In [195]:
# convert 'checkin' and 'checkout' to datetime
df['checkin'] = df['checkin'].astype('datetime64[ns]')
# checkout may be NaT, so we need to convert only if it is not NaT 
# entonces, usamos lambda para evitar el error
df['checkout'] = df['checkout'].apply(lambda x: x.astype('datetime64[ns]') if x is not NaT else None)

/tmp/ipykernel_10938/1161820117.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['checkin'] = df['checkin'].astype('datetime64[ns]')
/tmp/ipykernel_10938/1161820117.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.loc[df['checkout'].notnull(), 'checkout'] = df.loc[df['checkout'].notnull(), 'checkout'].astype('datetime64[ns]')


In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 15241 to 16726
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      38 non-null     datetime64[ns]
 1   name      38 non-null     object        
 2   dni       38 non-null     object        
 3   checkin   38 non-null     datetime64[ns]
 4   checkout  33 non-null     object        
dtypes: datetime64[ns](2), object(3)
memory usage: 1.8+ KB


In [197]:
df.head()

,date,name,dni,checkin,checkout
15241,2023-12-04,"Romero Leon, Maria Anton",06927369,2023-04-12 15:20:00,2023-04-12 23:04:00
15315,2023-12-05,"Romero Leon, Maria Anton",06927369,2023-05-12 15:19:00,2023-05-12 22:42:00
15411,2023-12-06,"Romero Leon, Maria Anton",06927369,2023-06-12 22:56:00,NaT
15483,2023-12-11,"Romero Leon, Maria Anton",06927369,2023-11-12 15:22:00,2023-11-12 23:51:00
15556,2023-12-12,"Romero Leon, Maria Anton",06927369,2023-12-12 15:14:00,2023-12-12 23:42:00


In [198]:
# create two columns: hour and minute representing the difference between checkin and checkout
# como checkout puede ser null, entonces la diferencia seria null mejora usamos lambda para evitar el error
df['hour'] = df.apply(lambda x: (x['checkout'] - x['checkin']).seconds // 3600 if x['checkout'] is not NaT else None, axis=1)
df['minute'] = df.apply(lambda x: ((x['checkout'] - x['checkin']).seconds % 3600) // 60 if x['checkout'] is not NaT else None, axis=1)

In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 15241 to 16726
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      38 non-null     datetime64[ns]
 1   name      38 non-null     object        
 2   dni       38 non-null     object        
 3   checkin   38 non-null     datetime64[ns]
 4   checkout  33 non-null     object        
 5   hour      33 non-null     float64       
 6   minute    33 non-null     float64       
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 2.4+ KB


In [200]:
df.head()

,date,name,dni,checkin,checkout,hour,minute
15241,2023-12-04,"Romero Leon, Maria Anton",06927369,2023-04-12 15:20:00,2023-04-12 23:04:00,7.0,44.0
15315,2023-12-05,"Romero Leon, Maria Anton",06927369,2023-05-12 15:19:00,2023-05-12 22:42:00,7.0,23.0
15411,2023-12-06,"Romero Leon, Maria Anton",06927369,2023-06-12 22:56:00,NaT,NaN,NaN
15483,2023-12-11,"Romero Leon, Maria Anton",06927369,2023-11-12 15:22:00,2023-11-12 23:51:00,8.0,29.0
15556,2023-12-12,"Romero Leon, Maria Anton",06927369,2023-12-12 15:14:00,2023-12-12 23:42:00,8.0,28.0


In [201]:
# queda formatear la fecha y hora
# el formato para la fecha es: dd/mm/yyyy
# el formato para la hora es: hh:mm:ss AM/PM
# para el formato de la hora usamos lambda para evitar el error
df['date'] = df['date'].apply(lambda x: x.strftime('%d/%m/%Y'))
df['checkin'] = df['checkin'].apply(lambda x: x.strftime('%I:%M:%S %p') if x is not NaT else None)
df['checkout'] = df['checkout'].apply(lambda x: x.strftime('%I:%M:%S %p') if x is not NaT else None)

In [202]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 15241 to 16726
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      38 non-null     object 
 1   name      38 non-null     object 
 2   dni       38 non-null     object 
 3   checkin   38 non-null     object 
 4   checkout  33 non-null     object 
 5   hour      33 non-null     float64
 6   minute    33 non-null     float64
dtypes: float64(2), object(5)
memory usage: 2.4+ KB


In [203]:
df.head()

,date,name,dni,checkin,checkout,hour,minute
15241,04/12/2023,"Romero Leon, Maria Anton",06927369,03:20:00 PM,11:04:00 PM,7.0,44.0
15315,05/12/2023,"Romero Leon, Maria Anton",06927369,03:19:00 PM,10:42:00 PM,7.0,23.0
15411,06/12/2023,"Romero Leon, Maria Anton",06927369,10:56:00 PM,None,NaN,NaN
15483,11/12/2023,"Romero Leon, Maria Anton",06927369,03:22:00 PM,11:51:00 PM,8.0,29.0
15556,12/12/2023,"Romero Leon, Maria Anton",06927369,03:14:00 PM,11:42:00 PM,8.0,28.0


In [204]:
# rename columns to spanish names in minuscule letters
df.rename(columns={'dni': 'dni', 'date': 'fecha', 'name': 'empleado', 'checkin': 'entrada', 'checkout': 'salida', 'hour': 'horas', 'minute': 'minutos'}, inplace=True)

In [207]:
df.head()

,fecha,empleado,dni,entrada,salida,horas,minutos
15241,04/12/2023,"Romero Leon, Maria Anton",06927369,03:20:00 PM,11:04:00 PM,7.0,44.0
15315,05/12/2023,"Romero Leon, Maria Anton",06927369,03:19:00 PM,10:42:00 PM,7.0,23.0
15411,06/12/2023,"Romero Leon, Maria Anton",06927369,10:56:00 PM,None,NaN,NaN
15483,11/12/2023,"Romero Leon, Maria Anton",06927369,03:22:00 PM,11:51:00 PM,8.0,29.0
15556,12/12/2023,"Romero Leon, Maria Anton",06927369,03:14:00 PM,11:42:00 PM,8.0,28.0


In [205]:
# group by employee(dni)
grouped = df.groupby('dni')

In [206]:
from pandas import ExcelWriter

# convert the group to excel, where each group is a sheet
# and the sheet name is the firstname and between parenthesis the dni
# the sheet name is limited to 31 characters

# create Excel file
writer = ExcelWriter('attendance.xlsx', engine='xlsxwriter')

# iterate over the groups
for name, group in grouped:
    # get the first name
    firstname = group['empleado'].iloc[0].split()[0]
    # get the dni
    dni = group['dni'].iloc[0]
    # create the sheet name
    sheet_name = f'{firstname} ({dni})'
    # if the sheet name is longer than 31 characters, truncate it
    if len(sheet_name) > 31:
        sheet_name = sheet_name[:31]
    # convert the group to excel
    group.to_excel(writer, sheet_name=sheet_name, index=False)

# save the Excel file
writer.close()